In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [92]:
pd.set_option('display.max_columns', None)

In [93]:
df = pd.read_csv('train_data_updated_v6.csv',parse_dates=['datetime'])

In [94]:
len(df)

1683767

In [95]:
df['target'].value_counts(dropna=False, normalize=True)

0.0    0.979786
1.0    0.016574
2.0    0.003640
Name: target, dtype: float64

В датасете много пропусков, заполним их нулями

In [96]:
df=df.fillna(0)

удалим ненужные признаки

In [97]:
cols=['lat_long', 'station', 'date', 'start_datetime', 'end_datetime']

In [33]:
df=df.drop(cols, axis=1)

In [35]:
data=df[df['target']==2]
data=data.append(df[df['target']==1])
data=data.append(df[df['target']==0])

In [36]:
data

,datetime,road_id,road_km,target,volume_1,volume_2,volume_3,volume_4,volume_5,volume_6,volume_7,volume_8,volume_9,volume_10,volume_11,volume_12,occupancy_1,occupancy_2,occupancy_3,occupancy_4,occupancy_5,occupancy_6,occupancy_7,occupancy_8,occupancy_9,occupancy_10,occupancy_11,occupancy_12,speed_1,speed_2,speed_3,speed_4,speed_5,speed_6,speed_7,speed_8,speed_9,speed_10,speed_11,speed_12,lane_count_f,lane_count_b,mean_volume_b,mean_volume_f,std_volume_b,std_volume_f,mean_occupancy_b,mean_occupancy_f,std_occupancy_b,std_occupancy_f,mean_speed_b,mean_speed_f,std_speed_b,std_speed_f,year,repair,avuch_loc,stabchar_type,cnt_subs,phenomenon,intensity,hour,weekday,year_season
2,2018-01-01 05:00:00,9,689,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,2018,1.0,0.0,0.0,0.0,0,0,5,0,1
3,2018-01-01 07:00:00,14,52,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,2018,0.0,0.0,0.0,0.0,0,0,7,0,1
5,2018-01-01 19:00:00,9,385,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,2018,0.0,0.0,0.0,0.0,0,0,19,0,1
6,2018-01-01 19:00:00,9,388,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,2018,0.0,0.0,0.0,0.0,0,0,19,0,1
15,2018-01-02 11:00:00,14,323,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,2018,0.0,0.0,0.0,0.0,0,0,11,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683762,2020-12-31 00:00:00,14,1285,0.0,33.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.859375,0.000000,0.0,0.0,0.0,0.0,0.496094,0.000000,0.0,0.0,0.0,0.0,64.97828,0.0000,0.0,0.0,0.0,0.0,81.545470,0.0000,0.0,0.0,0.0,0.0,1.0,1.0,33.0,19.0,0.0,0.0,0.859375,0.496094,0.000000,0.000000,64.97828,81.545470,0.00000,0.00000,2020,0.0,0.0,0.0,62.0,0,0,0,3,1
1683763,2020-12-31 00:00:00,14,1375,0.0,8.0,17.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.414062,1.390625,0.0,0.0,0.0,0.0,0.871094,0.000000,0.0,0.0,0.0,0.0,51.05000,47.0125,0.0,0.0,0.0,0.0,56.301876,0.0000,0.0,0.0,0.0,0.0,2.0,2.0,12.5,27.0,4.5,0.0,0.902344,0.871094,0.488281,0.000000,49.03125,56.301876,2.01875,0.00000,2020,0.0,0.0,0.0,100.0,0,0,0,3,1
1683764,2020-12-31 00:00:00,14,1381,0.0,85.0,6.0,0.0,0.0,0.0,0.0,66.0,15.0,0.0,0.0,0.0,0.0,1.066406,0.156250,0.0,0.0,0.0,0.0,1.242188,0.277344,0.0,0.0,0.0,0.0,87.18500,103.1325,0.0,0.0,0.0,0.0,81.165000,89.9775,0.0,0.0,0.0,0.0,2.0,2.0,45.5,40.5,39.5,25.5,0.611328,0.759766,0.455078,0.482422,95.15875,85.571250,7.97375,4.40625,2020,0.0,0.0,0.0,144.0,0,0,0,3,1
1683765,2020-12-31 00:00:00,14,1394,0.0,28.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.792969,0.000000,0.0,0.0,0.0,0.0,1.058594,0.000000,0.0,0.0,0.0,0.0,68.70469,0.0000,0.0,0.0,0.0,0.0,68.881096,0.0000,0.0,0.0,0.0,0.0,1.0,1.0,28.0,34.0,0.0,0.0,0.792969,1.058594,0.000000,0.000000,68.70469,68.881096,0.00000,0.0000

In [37]:
#вместо  лайбл инкодера
phen_index = data.groupby('phenomenon').agg({'target': 'sum'}).sort_values(by='target', ascending=False).index
phen_index = phen_index[::-1]
phen_dict = {}
for i, value in enumerate(phen_index):
    phen_dict[value] = i+1

data.phenomenon = data.phenomenon.map(phen_dict)
data.phenomenon = data.phenomenon.fillna(0)

inten_dict = {' слабая интенсивность': 1,
              ' умеренная интенсивность': 2,
              ' сильная интенсивность': 3}

data.intensity = data.intensity.map(inten_dict)
data.intensity = data.intensity.fillna(0)

In [38]:
len(data)

1683767

попробуем следующие модели:

In [42]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron

In [71]:
# model= ComplementNB(alpha=1,fit_prior= False)
model=PassiveAggressiveClassifier(n_jobs=-1, max_iter=3538, random_state=76,\
                                  class_weight={0.0:0.98, 1.:0.01, 2.: 0.004})
# model=Perceptron()
# model=SGDClassifier()

In [72]:
y=data.target.astype(float)
X=data.drop(['target', 'datetime'], axis=1).astype(float)

In [73]:
y

2          2.0
3          2.0
5          2.0
6          2.0
15         2.0
          ... 
1683762    0.0
1683763    0.0
1683764    0.0
1683765    0.0
1683766    0.0
Name: target, Length: 1683767, dtype: float64

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,  shuffle=True)

In [75]:
model.fit(X_train, y_train)

PassiveAggressiveClassifier(class_weight={0.0: 0.98, 1.0: 0.01, 2.0: 0.004},
                            max_iter=3538, n_jobs=-1, random_state=76)

In [76]:
pred=model.predict(X_test)

In [77]:
f1_score(y_test, pred, average='macro')

0.5658262936514621

количество значний по каждому классу не выглядит странно

In [78]:
pd.DataFrame(pred).value_counts()

0.0    547099
1.0      8014
2.0       531
dtype: int64

сделаем предсказание для тестовой выборки

In [79]:
test=pd.read_csv('test_data_v6.csv',parse_dates=['datetime'])

In [80]:
test

,datetime,road_id,road_km,target,volume_1,volume_2,volume_3,volume_4,volume_5,volume_6,volume_7,volume_8,volume_9,volume_10,volume_11,volume_12,occupancy_1,occupancy_2,occupancy_3,occupancy_4,occupancy_5,occupancy_6,occupancy_7,occupancy_8,occupancy_9,occupancy_10,occupancy_11,occupancy_12,speed_1,speed_2,speed_3,speed_4,speed_5,speed_6,speed_7,speed_8,speed_9,speed_10,speed_11,speed_12,lane_count_f,lane_count_b,mean_volume_b,mean_volume_f,std_volume_b,std_volume_f,mean_occupancy_b,mean_occupancy_f,std_occupancy_b,std_occupancy_f,mean_speed_b,mean_speed_f,std_speed_b,std_speed_f,year,repair,avuch_loc,stabchar_type,cnt_subs,lat_long,station,date,phenomenon,intensity,start_datetime,end_datetime,hour,weekday,year_season
0,2020-01-01 01:00:00,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1.0,NaN,NaN,107.0,"(55.76, 37.61)",MOSKMIH,2020-01-01,NaN,NaN,NaN,NaN,1,2,1
1,2020-01-01 01:00:00,9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1.0,2.0,3.0,107.0,"(55.76, 37.62)",MOSKMIH,2020-01-01,NaN,NaN,NaN,NaN,1,2,1
2,2020-01-01 01:00:00,9,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1.0,2.0,3.0,107.0,"(55.76, 37.63)",MOSKMIH,2020-01-01,NaN,NaN,NaN,NaN,1,2,1
3,2020-01-01 01:00:00,9,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1.0,2.0,3.0,107.0,"(55.77, 37.64)",MOSKMIH,2020-01-01,NaN,NaN,NaN,NaN,1,2,1
4,2020-01-01 01:00:00,9,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1.0,2.0,3.0,107.0,"(55.77, 37.63)",MOSKMIH,2020-01-01,NaN,NaN,NaN,NaN,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729446,2020-10-25 23:00:00,14,1545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,0.0,NaN,NaN,3.0,"(69.45, 30.74)",NIKEL,2020-10-25,NaN,NaN,NaN,NaN,23,6,4
1729447,2020-10-25 23:00:00,14,1546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,0.0,NaN,NaN,4.0,"(69.46, 30.72)",NIKEL,2020-10-25,NaN,NaN,NaN,NaN,23,6,4
1729448,2020-10-25 23:00:00,14,1547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,0.0,NaN,NaN,4.0,"(69.46, 30.7)",NIKEL,2020-10-25,NaN,NaN,NaN,NaN,23,6,4
1729449,2020-10-25 23:00:00,14,1548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,0.0,NaN,NaN,NaN,"(69.46, 30.68)",NIKEL,2020-10-25,NaN,NaN,NaN,NaN,23,6,4


преобразование тестовой выборки

In [81]:
test=test.fillna(0)
test=test.drop(['target', 'datetime'], axis=1)
test=test.drop(cols, axis=1)


In [82]:
# всесто лэйбл инкодера
test.phenomenon = test.phenomenon.map(phen_dict)
test.phenomenon = test.phenomenon.fillna(0)

test.intensity = test.intensity.map(inten_dict)
test.intensity = test.intensity.fillna(0)

In [ ]:
pred_test=model.predict(test)#предсказание

оформим красиво предсказание

In [84]:
test_frame=pd.read_csv('test.csv',parse_dates=['datetime'])

In [85]:
test_frame

,datetime,road_id,road_km,target
0,2020-01-01 01:00:00,9,0,NaN
1,2020-01-01 01:00:00,9,1,NaN
2,2020-01-01 01:00:00,9,2,NaN
3,2020-01-01 01:00:00,9,3,NaN
4,2020-01-01 01:00:00,9,4,NaN
...,...,...,...,...
1729446,2020-10-25 23:00:00,14,1545,NaN
1729447,2020-10-25 23:00:00,14,1546,NaN
1729448,2020-10-25 23:00:00,14,1547,NaN
1729449,2020-10-25 23:00:00,14,1548,NaN


In [88]:
test_frame['target']=pred_test

In [89]:
test_frame.target.value_counts()

0.0    1410521
1.0     223950
2.0      94980
Name: target, dtype: int64

In [90]:
test_frame.to_csv('submit.csv', index=None)